In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

from tasks.MCQA.mcqa import MCQA_task
datasets = MCQA_task.create_datasets(128)

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


In [3]:
from neural.pipeline import LMPipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pipeline = LMPipeline(model_name, max_new_tokens=1, device=device, dtype=torch.float16, max_length=64)
pipeline.tokenizer.padding_side = "left"

def checker(neural_output, causal_output):
    return causal_output in neural_output or neural_output in causal_output

print("DEVICE:", pipeline.model.device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

DEVICE: cuda:0


In [4]:
start = 0
end = pipeline.get_num_layers()
num_heads = pipeline.model.config.num_attention_heads
target_variables_list  = [["answer"], ["answer_position"]]
results_dir = "MCQA_attention_results"

In [5]:
# from experiments.attention_head_experiment import PatchAttentionHeads
# from neural.LM_units import get_all_tokens, TokenPosition


# for reg in [0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
#     config = {
#         "batch_size": 32, 
#         "evaluation_batch_size": 128, 
#         "training_epoch": 1, 
#         "n_features": 16, 
#         "regularization_coefficient": reg,
#         "featurizer_kwargs": {
#             "tie_masks": True
#         }
#     }

#     all_tokens = TokenPosition(lambda x: get_all_tokens(x, pipeline), pipeline, id="all_tokens")

#     heads_masking = [[(layer, head) for layer in range(0, end) for head in range(num_heads)]]
#     heads_patching =  [[(layer, head)] for layer in range(end-2, end) for head in range(num_heads)]

#     experiment = PatchAttentionHeads(pipeline, MCQA_task.causal_models["pointer"], heads_masking, all_tokens, checker, config=config)
        
#     experiment.train_interventions(datasets["random_counterfactual"], ["answer"], method="DBM", verbose=True)
#     raw_results = experiment.perform_interventions(datasets["random_counterfactual"], verbose=True, target_variables_list=target_variables_list, save_dir=results_dir+str(reg))
#     experiment.plot_mask_heatmap(raw_results, save_path=results_dir + str(reg))

In [6]:
from experiments.residual_stream_experiment import PatchResidualStream

config = {
    "batch_size": 64, 
    "evaluation_batch_size": 128, 
    "training_epoch": 4, 
    "n_features": 16, 
}

token_positions = MCQA_task.create_token_positions(pipeline)

experiment = PatchResidualStream(pipeline, MCQA_task.causal_models["pointer"], list(range(start, end)), list(token_positions.values()), checker, config=config)
# experiment.train_interventions(datasets, ["answer_position"], method="DAS", verbose=True)
raw_results = experiment.perform_interventions(datasets, verbose=False, target_variables_list=target_variables_list, save_dir=results_dir)
experiment.plot_heatmaps(raw_results, ["answer"], save_path=results_dir)
experiment.plot_heatmaps(raw_results, ["answer_position"], save_path=results_dir+"pointer")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
